## Preparation

users need to first import the jupyterlab_nodeeditor and specify the address of the YAML files

In [1]:
import jupyterlab_nodeeditor as jlne
from yggdrasil import yamlfile

In [2]:
sample_1 = "./sample_models/sample_1.yml"
photosynthesis =  "./sample_models/photosynthesis.yml"

## Workspace

In [3]:
editor=jlne.yml_trans([sample_1,photosynthesis])
editor

AppLayout(children=(Label(value='Node Editor', layout=Layout(grid_area='header')), Accordion(layout=Layout(gri…

## Export the YAML file

### Pre-processing Editor

In [4]:
def parse_editor_config(py_models_dict,editor_json):
    if "id" in editor_json.keys():
        node_ids=editor_json["nodes"].keys()
    else:
        node_ids=editor_json[list(editor_json.keys())[0]]["nodes"].keys()
    added_models=[]

    for node_id in node_ids:

        single_model_dict={}  #store a single model

        #extracted instance
        if "id" in editor_json.keys():
            inst=editor_json["nodes"][node_id]
        else:
            inst=editor_json[list(editor_json.keys())[0]]["nodes"][node_id]
        

        model_id=inst["id"]
        model_name=inst["name"]

        #store all the node_id and associated model name for later connection match
        inst_dict={}  
        inst_dict[model_id]=model_name


        input_ls=[]
        output_ls=[]

        inputs=list(py_models_dict[model_name]["inputs"])
        for model_input in inputs:
            single_port={}
            single_port["name"]=model_name+":"+model_input.title
            input_ls.append(single_port)

        outputs=list(py_models_dict[model_name]["outputs"])
        for model_output in outputs:
            single_port={}
            single_port["name"]=model_name+":"+model_output.title
            output_ls.append(single_port)


        single_model_dict["name"]=model_name
        if len(input_ls)!=0:
            single_model_dict["inputs"]=input_ls
        if len(output_ls)!=0:
            single_model_dict["outputs"]=output_ls

        #for test purpose
        single_model_dict["language"]="python"
        single_model_dict["args"]=py_models_dict[model_name]["args"]


        if model_name not in added_models:
            added_models.append(model_name)
            model_ls.append(single_model_dict)
    return(model_ls)

In [5]:
editor.node_editor.sync_config()
editor_json=editor.node_editor.editorConfig
editor_py=editor.node_editor.nodes


py_models_dict=dict()   # extract outputs/inputs/ args information from editor_py
for model_id in range(len(editor_py)):
    single_model=dict()
    single_model["args"]=editor_py[model_id].controls[0].value
    single_model["inputs"]=editor_py[model_id].inputs
    single_model["outputs"]=editor_py[model_id].outputs
    py_models_dict[editor_py[model_id].title]=single_model


yml_dict={} # the keywords for yml_dict include "models" and "connections"
model_ls=[]

if len(list(editor_json.keys()))==0:
    print("No component/instance has been detected in the workspace")
else:
    model_ls=parse_editor_config(py_models_dict,editor_json)  
    yml_dict["models"]=model_ls

### Dump YAML file

In [6]:
import yaml
stream = open("./sample_models/export_test.yml", 'w')
yaml.dump(yml_dict,stream)

## Import Test

In [7]:
export_test="./sample_models/export_test.yml"

In [8]:
# yamlfile.parse_yaml(export_test,model_only=True)

In [9]:
editor=jlne.yml_trans(export_test)
editor

AppLayout(children=(Label(value='Node Editor', layout=Layout(grid_area='header')), Accordion(layout=Layout(gri…